<a href="https://colab.research.google.com/github/karma-os/DeepLearningImplementation/blob/main/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alexnet 구현해보기


- CIFAR 10이나 Fashion-mnist 로 확장해보기
- 일단은 이대로 push


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Accuracy

#For optimizing
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import ReduceLROnPlateau



# Image Preprocessing 하는 augmentor 만들기

In [13]:
def alexnet_piepeline():
    """
    Initially, the pipeline flow should be . . .

    [1] Data Preprocessing :: ILSVRC data를 dataloader에 넣어서 data_generator 생성
        - augmentor는 albumentation의 horizontal_flip + randomcropresize + fancyPCA 이용

    [2] create_alexnet을 이용해서 AlexNet 모델을 생성
        - LRN layer는 BN layer로 대체

    [3] model compile, fit 시 
    optimizer=SGD(lr=0.01, momentum = 0.9, decay = 5 * 1e-4), loss='categorical_crossentropy', metrics=['accuracy']) 의 느낌으로 진행해야.

    #Learning Rate Scheduler
    rlr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=1, mode='min')

    """

    pass


def create_alexnet():
    """
    AlexNet consists of 5 blocks.

    [Feature Extraction Block]
    block 1,2: Convolution -> LRN -> Maxpooling
    block 3,4,5 : Convolution, with padding "same"

    [Classifier Block]
    Dropout -> Dense -> Dropout -> Dense

    issue:
    [0] Weight/ Bias Initializations, validation callbacks not applied
    [1] In PyTorch Implementation, input_size is different 
    [2] There is no LRN code in Keras. (PyTorch) 
        Used BatchNormalization instead.
    [3] I've not checked GPU implementation code and whether it is to check 
    [4] See Ref of pytorch libary


        -"One weird trick for parallelizing convolutional neural networks."
        (https://arxiv.org/abs/1404.5997)
    """

    input_tensor = Input(shape = (227,227,3), name="Alexnet_input")

    #Feature Extractor

    ##Block 1
    x = Conv2D(filters = 96, kernel_size = 11, strides=4, activation = "relu" ,name="conv1")(input_tensor)
    x = BatchNormalization(name= "Instead_LRN_1")(x)
    x = MaxPooling2D(pool_size=(3,3), strides = 2, name="pool1")(x)

    ## Block 2
    x = Conv2D(filters = 256, kernel_size= 5, activation="relu", padding="same", name="conv2")(x)
    x = BatchNormalization(name= "Instead_LRN_2")(x)
    x =  MaxPooling2D(pool_size = (2,2), name="pool2")(x)

    ## Block 3~5
    x = Conv2D(filters = 384, kernel_size = 3, activation = "relu", padding = "same", name = "conv3")(x)

    x = Conv2D(filters = 384, kernel_size = 3, activation = "relu", padding = "same", name = "conv4")(x) #이래야 사이즈 보존
 
    x = Conv2D(filters = 256, kernel_size = 3, activation = "relu", padding = "same", name = "conv5")(x)
 
    x = Flatten(name="Flatten")(x)


    #Classifier

    x = Dropout(rate = 0.5)(x)

    x = Dense(4096, name="FC1", activation = "relu")(x)

    x = Dropout(rate = 0.5, )(x)

    x = Dense(4096, name="FC2")(x)

    output = Dense(10, activation="softmax")(x)

    model = Model(inputs = input_tensor, outputs = output, name= "Alexnet_Practice")

    return model

model = create_alexnet()


Model: "Alexnet_Practice"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Alexnet_input (InputLayer)   [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 55, 55, 96)        34944     
_________________________________________________________________
Instead_LRN_1 (BatchNormaliz (None, 55, 55, 96)        384       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 27, 27, 96)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 27, 27, 256)       614656    
_________________________________________________________________
Instead_LRN_2 (BatchNormaliz (None, 27, 27, 256)       1024      
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 13, 13, 256)  